# Part I. ETL Pipeline for Pre-Processing the Files

## RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II.

## Now we can work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appears like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class' : 'SimpleStrategy',
     'replication_factor' : 1}"""
                   )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1:
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_library"
query = query + "(sessionId int, \
                itemInSession int,\
                song text,\
                artist text,\
                length float,\
                PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
# this dictionary simplifies the process of filling in the tables.
# we do not have to remember the ordering of the CSV file to access each information,
# but can access it via its name

dct = {
    'artist' : 0,
    'firstName' : 1,
    'gender' : 2,
    'itemInSession' : 3,
    'lastName' : 4,
    'length' : 5,
    'level' : 6,
    'location' : 7,
    'sessionId' : 8,
    'song' : 9,
    'userId' : 10,
}

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_library \
        (sessionId, itemInSession, song, artist, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (
            int(line[dct['sessionId']]),
            int(line[dct['itemInSession']]),
            line[dct['song']],
            line[dct['artist']],
            float(line[dct['length']])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = "SELECT artist, song, length FROM session_library \
        WHERE sessionId=338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df_result1 = pd.DataFrame(columns=[ 'Artist', 'Song', 'Length'])
for row in rows:
    df_result1 = df_result1.append({
        'Artist' : row.artist, 
        'Song' : row.song,
        'Length' : row.length
    }, ignore_index=True)
df_result1

,Artist,Song,Length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2:
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
query = "CREATE TABLE IF NOT EXISTS user_library"
query = query + "(userId int,\
                sessionId int, \
                itemInSession int,\
                song text,\
                artist text,\
                firstName text,\
                lastName text,\
                PRIMARY KEY (userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library \
        (userId, sessionId, itemInSession, song, artist, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (
            int(line[dct['userId']]),
            int(line[dct['sessionId']]),
            int(line[dct['itemInSession']]),
            line[dct['song']],
            line[dct['artist']],
            line[dct['firstName']],
            line[dct['lastName']]))

In [14]:
query = "SELECT artist, song, firstname, lastname FROM user_library \
        WHERE userId=10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    
df_result2 = pd.DataFrame(columns=[ 'Artist', 'Song', 'firstName', 'lastName'])
for row in rows:
    df_result2 = df_result2.append({
        'Artist' : row.artist,
        'Song' : row.song, 
        'firstName' : row.firstname,
        'lastName' : row.lastname,
    },ignore_index=True)
df_result2

,Artist,Song,firstName,lastName
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3:
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' primary key must be unique, so I add userId and sessionId, although not requested

In [15]:
query = "CREATE TABLE IF NOT EXISTS song_library"
query = query + "(song text,\
                firstName text,\
                lastName text,\
                userId int,\
                PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [16]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library \
        (song, firstName, lastName, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (
            line[dct['song']],
            line[dct['firstName']],
            line[dct['lastName']],
            int(line[dct['userId']])
        ))

In [17]:
## Query 3: Give me every user name (first and last) in my music app history 
# who listened to the song 'All Hands Against His Own'

query = "SELECT firstname, lastname FROM song_library \
        WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    
df_result3 = pd.DataFrame(columns=[ 'firstName', 'lastName'])
for row in rows:
    df_result3 = df_result3.append({
        'firstName' : row.firstname,
        'lastName' : row.lastname,
    },ignore_index=True)
df_result3

,firstName,lastName
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [18]:
query = "drop table session_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
query = "drop table song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()